In [84]:
#!pip install agentpy owlready2

In [85]:
import agentpy as ap
from matplotlib import pyplot as plt
import IPython
import random
from owlready2 import *

In [86]:
onto = get_ontology("file://onto.owl")

In [87]:
#ONTOLOGIA
#onto.delete()
with onto:

    class Entity(Thing):
        pass

    class Vaccum(Entity):
        pass

    class Dirt(Entity):
        pass

    class Place(Thing):
        pass

    class Position(Thing):
        pass

    class has_place(ObjectProperty, FunctionalProperty):
        domain = [Entity]
        range = [Place]

    class has_position(DataProperty, FunctionalProperty):
        domain = [Place]
        range = [str]

        pass



In [88]:
class VaccumAgent(ap.Agent):

    #Razonamiento
    def see(self,e):
        """
        Función de percepción
        @param e: entorno grid
        """
        self.per = []
        vecinos = e.neighbors(self,1)
        # Creación de percepto: lista de creencias de agentes Dirt
        for vecino in vecinos:
            self.per.append(Dirt(has_place=Place(has_position=str(e.positions[vecino]))))
        pass

    def next(self):
        """
        Función de razonamiento Deductivo
        """
        # Por cada acción
        for act in self.actions:
            # Por cada regla
            for rule in self.rules:
                # Si la acción es válidad dada la regla
                if rule(act):
                    # Ejecuta la acción
                    act()
        pass


    #REGLAS

    def rule_1(self,act):
        """
        Regla deductiva para limpiar
        @param act: acción a validar
        @return: booleano
        """
        # Validador de regla
        validador = [False, False]

        # Proposición 1: Si hay suciedad en la posición actual
        for dirty in self.model.dirties:
            if dirty.pos == self.model.grid.positions[self]:
                validador[0] = True

        # Proposición 2: Si la acción es la de limpiar
        if act == self.clean:
            validador[1] = True

        return sum(validador) == 2


    def rule_2(self,act):
        """
        Regla deductiva para moverse hacia el norte
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si hay suciedad en la posición Norte
        for dirty in self.per:
            if eval(dirty.has_place.has_position)[0] == self.model.grid.positions[self][0]-1:
                validador[0] = True

        # Proposición 2: Si la acción es la de moverse hacia el norte
        if act == self.move_N:
            validador[1] = True

        return sum(validador) == 2


    def rule_3(self,act):
        """
        Regla deductiva para moverse hacia el sur
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si hay suciedad en la posición Sur
        for dirty in self.per:
            if eval(dirty.has_place.has_position)[0] == self.model.grid.positions[self][0]+1:
                validador[0] = True

        # Proposición 2: Si la acción es la de moverse hacia el sur
        if act == self.move_S:
            validador[1] = True

        return sum(validador) == 2


    def rule_4(self,act):
        """
        Regla deductiva para moverse hacia el este
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si hay suciedad en la posición Este
        for dirty in self.per:
            if eval(dirty.has_place.has_position)[1] == self.model.grid.positions[self][1]+1:
                validador[0] = True

        # Proposición 2: Si la acción es la de moverse hacia el este
        if act == self.move_E:
            validador[1] = True

        return sum(validador) == 2


    def rule_5(self,act):
        """
        Regla deductiva para moverse hacia el oeste
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si hay suciedad en la posición Oeste
        for dirty in self.per:
            if eval(dirty.has_place.has_position)[1] == self.model.grid.positions[self][1]-1:
                validador[0] = True

        # Proposición 2: Si la acción es la de moverse hacia el oeste
        if act == self.move_W:
            validador[1] = True

        return sum(validador) == 2


    def rule_6(self,act):
        """
        Regla deductiva para moverse aleatoriamente
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si no hay suciedad en el entorno
        if len(self.per) <= 0:
            validador[0] = True

        # Proposición 2: Si la acción es la de moverse aleatoriamente
        if act == self.move_random:
            validador[1] = True

        return sum(validador) == 2


    #SIMULACIÓN DE AGENTE

    def setup(self):
        """
        Función de inicialización
        """

        self.agentType = 0 # Tipo de agente
        self.direction = (-1,0) # Dirección inicial
        # Acciones del agente
        self.actions = (
            self.clean,
            self.move_N,
            self.move_S,
            self.move_E,
            self.move_W,
            self.move_random
        )
        # Reglas del agente
        self.rules = (
            self.rule_1,
            self.rule_2,
            self.rule_3,
            self.rule_4,
            self.rule_5,
            self.rule_6
        )
        pass

    def step(self):
        """
        Función paso a paso
        """
        self.see(self.model.grid) #Percepción del Grid
        self.next() #Razonamiento y acción
        pass

    def update(self):
        pass

    def end(self):
        pass

    #ACCIONES
    def move_N(self):
        """
        Función de movimiento hacia el norte
        """
        self.direction = (-1,0) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_S(self):
        """
        Función de movimiento hacia el sur
        """
        self.direction = (1,0) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_E(self):
        """
        Función de movimiento hacia el este
        """
        self.direction = (0,1) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_W(self):
        """
        Función de movimiento hacia el oeste
        """
        self.direction = (0,-1) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_random(self):
        """
        Función de movimiento aleatorio
        """
        # Rotaciones aleatorias
        for _ in range(random.randint(0,4)):
            self.turn()
        self.forward() # Caminar un paso hacia adelante

    def clean(self):
        """
        Función de limpieza
        """
        #Si hay suciedad en la posición actual
        for dirty in self.model.grid.agents:
          if dirty.agentType == 1:
            if dirty.pos == self.model.grid.positions[self]:
              self.model.grid.remove_agents(dirty) #Eliminación de suciedad
              break #Romper ciclo
        pass

    def forward(self):
        """
        Función de movimiento
        """
        self.model.grid.move_by(self, self.direction)
        pass

    def turn(self):
        """
        Función de rotación
        """
        if self.direction == (-1,0):
            self.direction = (0,1) #Hacia Este
        elif self.direction == (0,1):
            self.direction = (1,0)  #Hacia Sur
        elif self.direction == (1,0):
            self.direction = (0,-1) #Hacia Oeste
        elif self.direction == (0,-1):
            self.direction = (-1,0) #Hacia Norte
        pass

In [89]:
class DirtAgent(ap.Agent):

    def setup(self):
        self.agentType = 1
        self.first_step = True
        self.pos = None
        pass

    def step(self):
        if self.first_step:
          self.pos = self.model.grid.positions[self]
          self.first_step = False
        pass

    def update(self):
        pass

    def end(self):
        pass

In [90]:
class VaccumModel(ap.Model):

    def setup(self):
        """
        Función de inicialización
        """
        # Instancias lista de agentes aspiradoras
        self.vaccums = ap.AgentList(self, self.p.vaccums, VaccumAgent)
        # Instancias lista de agentes suciedad
        self.dirties = ap.AgentList(self, self.p.dirties, DirtAgent)

        # Instancia grid
        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)

        # Asignación de agentes a grid
        self.grid.add_agents(self.vaccums, random=True, empty=True)
        self.grid.add_agents(self.dirties, random=True, empty=True)
        pass

    def step(self):
        """
        Función paso a paso
        """
        self.vaccums.step() #Paso de aspiradora
        self.dirties.step() #Paso de suciedad
        pass

    def update(self):
        pass

    def end(self):
        pass

In [91]:
parameters = {
    'M': 10,
    'N': 10,
    "steps": 100,
    'vaccums': 1,
    'dirties': 10
}

#model = VaccumModel(parameters)
#results = model.run()

In [92]:
#A FUNCTION TO ANIMATE THEE SIMULATION

def animation_plot(model, ax):
    """
    Función de animación
    @param model: modelo
    @param ax: axes (matplotlib)
    """
    # Definición de atributo para tipo de agente
    agent_type_grid = model.grid.attr_grid('agentType')
    # Definición de gráfico con colores (de acuerdo al tipo de agente)
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    # Definición de título del gráfico
    ax.set_title(f"Vaccum Model \n Time-step: {model.t}, "
                 f"Drity: {0}")

In [93]:
#SIMULATION:

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = VaccumModel(parameters)

#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())